# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment, Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.widgets import RunDetails

import joblib

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
experiment_name = 'hypertuning-experiment'

experiment=Experiment(ws, experiment_name)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code AYY4LZFLX to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.


### Compute Cluster

In [4]:
# Create compute cluster
from azureml.core.compute_target import ComputeTargetException
# Use vm_size = "STANDARD_DS2_V2" 

cpu_cluster_name = 'hypertuning'
try:
    compute_target = ComputeTarget(workspace=ws,name=cpu_cluster_name)
    print('existing cluster found and will be used')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS12_v2', max_nodes=4)
    compute_target = ComputeTarget.create(ws,cpu_cluster_name,compute_config)
    compute_target.wait_for_completion(show_output=True)

InProgress....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [8]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice, normal
import os

# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({'--C': uniform(0.001,0.5),"--max_iter":choice(10,100,1000,10000)})

#TODO: Create your estimator and hyperdrive config
estimator = SKLearn(source_directory='.',compute_target=compute_target, entry_script='train.py')

hyperdrive_run_config = HyperDriveConfig(estimator=estimator, hyperparameter_sampling=param_sampling, policy=early_termination_policy, max_concurrent_runs=4, primary_metric_name='Accuracy',max_total_runs=30,primary_metric_goal=PrimaryMetricGoal.MAXIMIZE)
# https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.hyperdrive.hyperdriveconfig?view=azure-ml-py

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [9]:
#TODO: Submit your experiment
run_project_hypertuning = experiment.submit(config=hyperdrive_run_config, show_output=True, wait_for_completion=True)


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [10]:
RunDetails(run_project_hypertuning).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [11]:
best_run = run_project_hypertuning.get_best_run_by_primary_metric()
print(run_project_hypertuning)
print('run ID: ', best_run.id)
print('best metric: ', best_run.get_metrics())
print('hyperparams: ', best_run.get_details())

Run(Experiment: hypertuning-experiment,
Id: HD_848aab14-9147-496b-b04f-00048405bfef,
Type: hyperdrive,
Status: Completed)
run ID:  HD_848aab14-9147-496b-b04f-00048405bfef_2
best metric:  {'Regularization Strength:': 0.07970755084992351, 'Max iterations:': 1000, 'Accuracy': 0.5615384615384615}
hyperparams:  {'runId': 'HD_848aab14-9147-496b-b04f-00048405bfef_2', 'target': 'hypertuning', 'status': 'Completed', 'startTimeUtc': '2021-11-22T15:11:57.918762Z', 'endTimeUtc': '2021-11-22T15:11:59.231578Z', 'services': {}, 'warnings': [{'message': 'This run might be using a new job runtime with improved performance and error reporting. The logs from your script are in user_logs/std_log.txt. Please let us know if you run into any issues, and if you would like to opt-out, please add the environment variable AZUREML_COMPUTE_USE_COMMON_RUNTIME to the environment variables section of the job and set its value to the string "false"'}], 'properties': {'_azureml.ComputeTargetType': 'amlcompute', 'Conten

In [13]:
print(best_run)

Run(Experiment: hypertuning-experiment,
Id: HD_848aab14-9147-496b-b04f-00048405bfef_2,
Type: azureml.scriptrun,
Status: Completed)


In [29]:
# TODO save best model
registered_model = best_run.register_model(model_name="hypertuning",model_path='.',properties={'accuracy': best_run.get_metrics()['Accuracy'], 'regularization strength': best_run.get_metrics()['Regularization Strength:'], 'maximum iterations': best_run.get_metrics()['Max iterations:']})
print(registered_model)

Model(workspace=Workspace.create(name='quick-starts-ws-164203', subscription_id='b968fb36-f06a-4c76-a15f-afab68ae7667', resource_group='aml-quickstarts-164203'), name=hypertuning, id=hypertuning:4, version=4, tags={}, properties={'accuracy': '0.5615384615384615', 'regularization strength': '0.07970755084992351', 'maximum iterations': '1000'})


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

